In [ ]:
import multiprocessing as mp

def compute_block(dp, str1, str2, i_start, i_end):
    """ฟังก์ชันคำนวณช่วงของบล็อกใน dp array"""
    m, n = len(str1), len(str2)

    for i in range(i_start, i_end):
        for j in range(1, n + 1):
            if i == 0:
                dp[i][j] = j
            elif j == 0:
                dp[i][j] = i
            elif str1[i - 1] == str2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = 1 + min(dp[i][j - 1], dp[i - 1][j], dp[i - 1][j - 1])

def edit_distance_parallel(str1, str2):
    m, n = len(str1), len(str2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    
    # คำนวณค่าต่างๆ ใน dp array แบบขนานโดยแบ่งเป็นกลุ่ม
    num_workers = mp.cpu_count()  # ใช้จำนวน CPU ที่สามารถใช้ได้
    pool = mp.Pool(num_workers)
    
    # กำหนดขอบเขตของบล็อกการทำงาน
    block_size = m // num_workers
    tasks = []

    for i in range(num_workers):
        i_start = i * block_size
        i_end = (i + 1) * block_size if i < num_workers - 1 else m + 1
        tasks.append(pool.apply_async(compute_block, (dp, str1, str2, i_start, i_end)))

    # รอให้การทำงานขนานทั้งหมดเสร็จสิ้น
    for task in tasks:
        task.get()

    pool.close()
    pool.join()

    return dp[m][n]

# ระบุเส้นทางไฟล์ของคุณ
file_path1 = "C:\\Users\\BMEI CMU\\Documents\\DikeASCII_DATASET\\Benign\\output1.txt"
file_path2 = "C:\\Users\\BMEI CMU\\Documents\\DikeASCII_DATASET\\Benign\\output2.txt"

# อ่านเนื้อหาไฟล์แรก
with open(file_path1, 'r', encoding='utf-8') as file1:
    content1 = file1.read()

# อ่านเนื้อหาไฟล์ที่สอง
with open(file_path2, 'r', encoding='utf-8') as file2:
    content2 = file2.read()

# คำนวณ Edit Distance แบบขนาน
distance = edit_distance_parallel(content1, content2)

print(f"Edit Distance between the contents of file 1 and file 2: {distance}")
